In [13]:
import pandas as pd

# Use the file name as it appears in your Colab 'Files' sidebar
# If it is inside a folder named 'archive', use: 'archive/customer_support_tickets.csv'
file_path = '/customer_support_tickets.csv'

df = pd.read_csv(file_path)

# Task 1: Explore the dataset
print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()

Dataset loaded with 8469 rows and 17 columns.


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [14]:
# Task 2: Data Cleaning
# Converting the Ticket Description to lowercase and removing special characters
import re

# We use the column name 'Ticket Description' from your dataset
df['Cleaned_Description'] = df['Ticket Description'].str.lower()

# Remove special characters using a simple regular expression
df['Cleaned_Description'] = df['Cleaned_Description'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))

# Remove extra spaces
df['Cleaned_Description'] = df['Cleaned_Description'].str.strip()

print("Task 2 Complete: Messages cleaned!")
df[['Ticket Description', 'Cleaned_Description']].head()

Task 2 Complete: Messages cleaned!


,Ticket Description,Cleaned_Description
0,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
1,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
2,I'm facing a problem with my {product_purchase...,im facing a problem with my productpurchased t...
3,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
4,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...


In [19]:
# Task 3: Classification Logic (Defining the function)
def classify_ticket(text):
    text = str(text)
    if any(word in text for word in ['pay', 'bill', 'charge', 'invoice', 'card']):
        return 'PAYMENT'
    elif any(word in text for word in ['login', 'password', 'account', 'access', 'sign']):
        return 'LOGIN'
    elif any(word in text for word in ['delivery', 'shipping', 'track', 'order', 'package']):
        return 'DELIVERY'
    elif any(word in text for word in ['refund', 'money back', 'return']):
        return 'REFUND'
    elif any(word in text for word in ['bug', 'error', 'crash', 'not working', 'issue']):
        return 'BUG'
    else:
        return 'GENERAL'

# Task 3: Applying the function (This is where the error was)
df['Category'] = df['Cleaned_Description'].apply(classify_ticket)

# Check the results
print("Task 3 Complete: Tickets Categorized!")
df[['Cleaned_Description', 'Category']].head(10)

Task 3 Complete: Tickets Categorized!


,Cleaned_Description,Category
0,im having an issue with the productpurchased p...,PAYMENT
1,im having an issue with the productpurchased p...,BUG
2,im facing a problem with my productpurchased t...,PAYMENT
3,im having an issue with the productpurchased p...,BUG
4,im having an issue with the productpurchased p...,DELIVERY
5,im facing a problem with my productpurchased t...,GENERAL
6,im unable to access my productpurchased accoun...,LOGIN
7,im having an issue with the productpurchased p...,BUG
8,im having an issue with the productpurchased p...,BUG
9,my productpurchased is making strange noises a...,BUG


In [20]:
# Task 4: Priority Assignment Logic
def assign_priority(row):
    category = row['Category']

    # P0: Critical - Business is losing money or users can't get in
    if category == 'PAYMENT' or category == 'LOGIN':
        return 'P0'
    # P1: High - Something is broken (Bugs/Errors)
    elif category == 'BUG':
        return 'P1'
    # P2: Medium - Shipping and returns
    elif category == 'DELIVERY' or category == 'REFUND':
        return 'P2'
    # P3: Low - General questions
    else:
        return 'P3'

# Apply the priority assignment to the dataframe
df['Priority'] = df.apply(assign_priority, axis=1)

print("Task 4 Complete: Priorities Assigned!")
# Show the results to verify
df[['Category', 'Priority']].head(10)

Task 4 Complete: Priorities Assigned!


,Category,Priority
0,PAYMENT,P0
1,BUG,P1
2,PAYMENT,P0
3,BUG,P1
4,DELIVERY,P2
5,GENERAL,P3
6,LOGIN,P0
7,BUG,P1
8,BUG,P1
9,BUG,P1


In [21]:
# Task 5: SLA Calculation
import pandas as pd
from datetime import timedelta

# 1. Define SLA hours based on Priority
# P0 = 4 hours, P1 = 24 hours, P2 = 48 hours, P3 = 72 hours
priority_sla = {
    'P0': 4,
    'P1': 24,
    'P2': 48,
    'P3': 72
}

# 2. Map the SLA hours to each ticket
df['SLA_Hours'] = df['Priority'].map(priority_sla)

# 3. Convert 'Date of Purchase' to datetime (assuming this is your start time)
# Note: In a real system, you'd use 'Ticket Created Time', but let's use Purchase Date for this lab.
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'])

# 4. Calculate the Due Date
df['Due_Date'] = df['Date of Purchase'] + pd.to_timedelta(df['SLA_Hours'], unit='h')

print("Task 5 Complete: SLA and Due Dates calculated!")
df[['Priority', 'SLA_Hours', 'Date of Purchase', 'Due_Date']].head(10)

Task 5 Complete: SLA and Due Dates calculated!


,Priority,SLA_Hours,Date of Purchase,Due_Date
0,P0,4,2021-03-22,2021-03-22 04:00:00
1,P1,24,2021-05-22,2021-05-23 00:00:00
2,P0,4,2020-07-14,2020-07-14 04:00:00
3,P1,24,2020-11-13,2020-11-14 00:00:00
4,P2,48,2020-02-04,2020-02-06 00:00:00
5,P3,72,2020-07-28,2020-07-31 00:00:00
6,P0,4,2020-02-23,2020-02-23 04:00:00
7,P1,24,2020-08-09,2020-08-10 00:00:00
8,P1,24,2020-07-16,2020-07-17 00:00:00
9,P1,24,2020-03-06,2020-03-07 00:00:00


In [22]:
# Task 6: Generate Report and Export CSV
print("--- SUPPORT MANAGER REPORT ---")

# 1. Summary by Category
print("\nTickets by Category:")
print(df['Category'].value_counts())

# 2. Summary by Priority
print("\nTickets by Priority:")
print(df['Priority'].value_counts())

# 3. Export the final results to a new CSV file
output_filename = 'final_customer_support_triage.csv'
df.to_csv(output_filename, index=False)

print(f"\n--- SUCCESS ---")
print(f"Final report generated and data saved to {output_filename}")

--- SUPPORT MANAGER REPORT ---

Tickets by Category:
Category
BUG         6032
LOGIN        783
PAYMENT      610
DELIVERY     552
GENERAL      307
REFUND       185
Name: count, dtype: int64

Tickets by Priority:
Priority
P1    6032
P0    1393
P2     737
P3     307
Name: count, dtype: int64

--- SUCCESS ---
Final report generated and data saved to final_customer_support_triage.csv
